This notebook will extract information from all of our DDOS csv files and place information into a large samples/features matrix. The matrix will look something like:
```
    SOURCE_IP | DEST_IP | <Protocol>TRAFFIC_FROM_SRC_TO_DEST
```
Where `<Protocol>` can be ICMP, UDP, TCP, and a few others.

In [1]:
import pandas as pd 
import numpy as np 
import csv as csv 
import glob 
import pickle
from datetime import datetime 
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
to_drop = ['<->', 'rel_start']
names = ['src', '<->', 'dest', 'in_frames', 'in_bytes', 'out_frames', 'out_bytes', 'total_frames', 'total_bytes', 'rel_start', 'duration']

In [6]:
# Begin dataframe by importing the total data, which will have all IPs
protocol_names = ['total' + '_' + name for name in names]
protocol_to_drop = ['total' + '_' + name for name in to_drop]
passive_df = pd.read_csv('../smallpassive_total.tshark.csv', header=None, skiprows=5, names=protocol_names)
passive_df.drop(passive_df.tail(1).index)

,total_src,total_<->,total_dest,total_in_frames,total_in_bytes,total_out_frames,total_out_bytes,total_total_frames,total_total_bytes,total_rel_start,total_duration
0,220.20.200.164,<->,84.236.193.94,21112,31747024,0,0,21112,31747024,0.000003,1.9944
1,109.97.67.71,<->,92.27.27.185,0,0,7760,6072145,7760,6072145,0.004502,1.9954
2,247.136.134.84,<->,22.249.49.193,5769,8676576,0,0,5769,8676576,0.003426,1.9944
3,220.28.73.211,<->,215.158.140.162,0,0,4008,5811949,4008,5811949,0.277408,1.6792
4,121.23.5.158,<->,89.183.104.248,0,0,3799,4718748,3799,4718748,0.002621,1.9953
5,144.93.121.86,<->,144.93.120.177,3414,696456,0,0,3414,696456,0.001075,1.9987
6,215.158.238.110,<->,56.20.122.75,3346,5026059,0,0,3346,5026059,0.000020,1.9117
7,145.19.13.114,<->,14.27.112.65,3258,4900032,0,0,3258,4900032,0.010939,1.9877
8,133.178.226.214,<->,100.224.243.106,3137,4515913,0,0,3137,4515913,1.033264,0.9531
9,192.228.188.15,<->,133.178.227.105,0,0,2986,4362558,2986,4362558,0.574513,0.6483
